In [ ]:
%pwd

In [23]:
file_name = 'float_trace.txt'
def analyze_trace(file_name):
    total = 0
    out_file = open('pread64_trace_'+file_name, 'w')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    result = {}
    for line in lines:
        if line.startswith('pread64'):
            size = int(line.split('=')[-1].strip())
            pos = int(line.split('=')[-2].split(', ')[-1].replace(")",""))
            # print(pos, size)
            out_file.write(str(pos)+' '+str(size)+'\n')
            total += size
            if size in result:
                result[size] += 1
            else:
                result[size] = 1
    return dict(sorted(result.items())), total
    # return dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

In [24]:
files = ['float_trace.txt', 'string_trace.txt', 'int_trace.txt']
for f in files:
    result, total = analyze_trace(f)
    print(f, total)

float_trace.txt 156109113
string_trace.txt 106598104
int_trace.txt 39901415


In [25]:
files = ['float_trace_pq.txt', 'string_trace_pq.txt', 'int_trace_pq.txt']
for f in files:
    result, total = analyze_trace(f)
    print(f, total)

float_trace_pq.txt 51364590
string_trace_pq.txt 108423725
int_trace_pq.txt 37656438


In [28]:
import os
os.system('sync; echo 3 | sudo tee /proc/sys/vm/drop_caches')
for t in ['float', 'string', 'int']:
    print(f'type: {t}')
    print('orc      pq')
    orc_time = os.popen(f'./repro pread64_trace_{t}_trace.txt {t}_core_UNCOMPRESSED_speed_1_2147486647.orc').read().strip('\n')
    pq_time = os.popen(f'./repro pread64_trace_{t}_trace_pq.txt {t}_core_NONE_True_2147483647_2.6_2.0.parquet').read().strip('\n')
    print(f'{orc_time} {pq_time}')

3
type: float
orc      pq
97545 86545
type: string
orc      pq
160570 188001
type: int
orc      pq
31718 64934
